User with Most Approved Flags

Which user flagged the most distinct videos that ended up approved by YouTube? Output, in one column, their full name or names in case of a tie. In the user's full name, include a space between the first and the last name.

In [1]:
import pandas as pd

In [2]:
user_flags = pd.read_excel("../CSV/user_flags.xlsx", skiprows=1)
user_flags.head()

,user_firstname,user_lastname,video_id,flag_id
0,Richard,Hasson,y6120QOlsfU,0cazx3
1,Mark,May,Ct6BUPvE2sM,1cn76u
2,Gina,Korman,dQw4w9WgXcQ,1i43zk
3,Mark,May,Ct6BUPvE2sM,1n0vef
4,Mark,May,jNQXAC9IVRw,1sv6ib


In [4]:
flag_review = pd.read_excel("../CSV/flag_review.xlsx", skiprows=1)
flag_review.head()

,flag_id,reviewed_by_yt,reviewed_date,reviewed_outcome
0,0cazx3,False,NaN,NaN
1,1cn76u,True,2022-03-15,REMOVED
2,1i43zk,True,2022-03-15,REMOVED
3,1n0vef,True,2022-03-15,REMOVED
4,1sv6ib,True,2022-03-15,APPROVED


In [7]:
result = user_flags.merge(right=flag_review, how="inner", on="flag_id")
result.head()

,user_firstname,user_lastname,video_id,flag_id,reviewed_by_yt,reviewed_date,reviewed_outcome
0,Richard,Hasson,y6120QOlsfU,0cazx3,False,NaN,NaN
1,Mark,May,Ct6BUPvE2sM,1cn76u,True,2022-03-15,REMOVED
2,Gina,Korman,dQw4w9WgXcQ,1i43zk,True,2022-03-15,REMOVED
3,Mark,May,Ct6BUPvE2sM,1n0vef,True,2022-03-15,REMOVED
4,Mark,May,jNQXAC9IVRw,1sv6ib,True,2022-03-15,APPROVED


In [8]:
result = result[result["reviewed_outcome"].str.lower() == "approved"]
result.head()

,user_firstname,user_lastname,video_id,flag_id,reviewed_by_yt,reviewed_date,reviewed_outcome
4,Mark,May,jNQXAC9IVRw,1sv6ib,True,2022-03-15,APPROVED
6,Mark,May,5qap5aO4i9A,4cvwuv,True,2022-03-15,APPROVED
8,Richard,Hasson,y6120QOlsfU,6jjkvn,True,2022-03-16,APPROVED
9,Pauline,Wilks,jNQXAC9IVRw,7ks264,True,2022-03-15,APPROVED
12,Richard,Hasson,dQw4w9WgXcQ,arydfd,True,2022-03-15,APPROVED


In [9]:
result["username"] = result["user_firstname"] + " " + result["user_lastname"]
result

,user_firstname,user_lastname,video_id,flag_id,reviewed_by_yt,reviewed_date,reviewed_outcome,username
4,Mark,May,jNQXAC9IVRw,1sv6ib,True,2022-03-15,APPROVED,Mark May
6,Mark,May,5qap5aO4i9A,4cvwuv,True,2022-03-15,APPROVED,Mark May
8,Richard,Hasson,y6120QOlsfU,6jjkvn,True,2022-03-16,APPROVED,Richard Hasson
9,Pauline,Wilks,jNQXAC9IVRw,7ks264,True,2022-03-15,APPROVED,Pauline Wilks
12,Richard,Hasson,dQw4w9WgXcQ,arydfd,True,2022-03-15,APPROVED,Richard Hasson
14,Richard,Hasson,dQw4w9WgXcQ,ehn1pt,True,2022-03-18,APPROVED,Richard Hasson
19,William,Kwan,y6120QOlsfU,kktiwe,True,2022-03-14,APPROVED,William Kwan
21,Pauline,Wilks,jNQXAC9IVRw,ov5gd8,True,2022-03-17,APPROVED,Pauline Wilks
23,Daniel,Bell,5qap5aO4i9A,xciyse,True,2022-03-16,APPROVED,Daniel Bell
24,Evelyn,Johnson,dQw4w9WgXcQ,xvhk6d,True,2022-03-17,APPROVED,Evelyn Johnson


In [10]:
result = result.groupby(by="username")["video_id"].nunique().reset_index()
result

,username,video_id
0,Daniel Bell,1
1,Evelyn Johnson,1
2,Mark May,2
3,Pauline Wilks,1
4,Richard Hasson,2
5,William Kwan,1


In [11]:
result["rank"] = result["video_id"].rank(method="dense", ascending=False)
result

,username,video_id,rank
0,Daniel Bell,1,2.0
1,Evelyn Johnson,1,2.0
2,Mark May,2,1.0
3,Pauline Wilks,1,2.0
4,Richard Hasson,2,1.0
5,William Kwan,1,2.0


In [12]:
result = result[result["rank"] == 1]["username"]
result

2          Mark May
4    Richard Hasson
Name: username, dtype: object

Solution Walkthrough
This walkthrough will guide you through the code to find the user who flagged the most distinct videos that were approved by YouTube. The code performs several steps to reach the final result.

Understanding The Data
Before diving into the code, let's understand the data involved in this problem. We have two tables: user_flags and flag_review.

The user_flags table contains information about the flags raised by users. It has the following columns:

flag_id: an identifier for each flag
user_firstname: the first name of the user who flagged the video
user_lastname: the last name of the user who flagged the video
video_id: the identifier of the flagged video
The flag_review table contains information about the reviews of the flagged videos by YouTube. It has the following columns:

flag_id: an identifier for each flag (same as in user_flags)
reviewed_outcome: the outcome of the review, whether it was approved or not
The Problem Statement
The task at hand is to find the user or users who flagged the most distinct videos that ended up being approved by YouTube. In case of a tie, we need to include all the users in the output.

Breaking Down The Code
Let's break down the provided code into smaller components to better understand its functionality.

result = user_flags.merge(right=flag_review, how="inner", on="flag_id")

This code merges the user_flags and flag_review tables on the common column "flag_id" using an inner join. The result is stored in the result variable.
result = result[result["reviewed_outcome"].str.lower() == "approved"]

This code filters out the rows in the result table where the "reviewed_outcome" column has the value "approved". Only the rows with approved videos remain in the result table.
result["username"] = result["user_firstname"] + " " + result["user_lastname"]

This code creates a new column "username" in the result table by concatenating the "user_firstname" and "user_lastname" columns with a space in between.
result = result.groupby(by="username")["video_id"].nunique().reset_index()

This code groups the rows in the result table by the "username" column and calculates the number of unique "video_id" values for each username. The result is stored in the result variable, replacing the previous value.
result["rank"] = result["video_id"].rank(method="dense", ascending=False)

This code adds a new column "rank" to the result table, which contains the ranking of each username based on the number of distinct videos flagged. The ranking is calculated in descending order.
result = result[result["rank"] == 1]["username"]

This code filters out the rows in the result table where the "rank" column has the value 1. It selects the usernames corresponding to the top-ranked users who flagged the most distinct videos.
Bringing It All Together
The complete code performs the following steps in sequence:

Merges the user_flags and flag_review tables based on the "flag_id" column using an inner join.
Filters out the rows where the video reviews were not "approved".
Creates a new column "username" by concatenating the first name and last name of the users.
Groups the remaining rows by "username" and calculates the number of unique videos flagged by each user.
Ranks the users based on the number of distinct videos flagged, in descending order.
Selects the usernames of the top-ranked users who flagged the most distinct videos (with a rank of 1).
Conclusion
The code successfully finds the user or users who flagged the most distinct videos that ended up being approved by YouTube. It outputs their full names in case of a tie.